In [1]:
from lib import *

#### Levanto datos y armo velas de volumen

In [2]:
#from lib import DataHandler, FeaturesGenerator, Labeler
# Inicializo
dh = DataHandler("./data.csv", 
                 tick_data=False)
# Genero diferentes data con volume bars
vv_100000 = dh.generate_volumebars(100000)
vv_500000 = dh.generate_volumebars(500000)
print(vv_100000.head(),
      vv_500000.head() )


                        open     high      low    close  volume
datetime                                                       
2017-01-02 17:03:00  2364.50  2376.75  2361.75  2373.25   96576
2017-01-03 08:31:00  2373.25  2375.50  2369.75  2372.00  102012
2017-01-03 09:00:00  2371.75  2381.00  2371.50  2378.50  101353
2017-01-03 09:30:00  2378.75  2379.00  2365.50  2368.50   99640
2017-01-03 10:12:00  2368.50  2370.00  2361.00  2364.00   99626                        open     high      low   close  volume
datetime                                                     
2017-01-02 17:03:00  2364.5  2381.00  2361.00  2364.0  499207
2017-01-03 10:58:00  2364.0  2382.00  2361.25  2382.0  494166
2017-01-04 08:32:00  2382.0  2388.75  2379.00  2386.0  503263
2017-01-04 14:53:00  2386.0  2387.50  2375.50  2380.0  503198
2017-01-05 11:54:00  2380.0  2388.25  2379.50  2384.0  500070


#### Genero los features

In [3]:
# Genero features para cada data por separado
fg = FeaturesGenerator(vv_100000, 
                         momentum=True, 
                         volume=False, 
                         volatility=False)
ff_100000 = fg.df

fg = FeaturesGenerator(vv_500000, 
                         momentum=True, 
                         volume=False, 
                         volatility=False)
ff_500000 = fg.df


In [10]:
ff_100000.head()

,AO_5_34,BOP,AR_26,BR_26,CCI_14_0.015,CFO_9,CG_10,CMO_14,COPC_11_14_10,CTI_12,...,STOCHk_14_3_3,STOCHd_14_3_3,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,TRIX_30_9,TRIXs_30_9,TSI_13_25_13,TSIs_13_25_13,UO_7_14_28,WILLR_14
datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-02 17:03:00,NaN,0.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 08:31:00,NaN,-0.217391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 09:00:00,NaN,0.710526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 09:30:00,NaN,-0.759259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03 10:12:00,NaN,-0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Genero los labels

In [4]:
lb_100000 = Labeler(vv_100000,
                    horizons=[50, 100],
                    betas=[1.5, 2],
                    periods=[14, 28])

lb_500000 = Labeler(vv_500000,
                    horizons=[50, 100],
                    betas=[1.5, 2],
                    periods=[14, 28])


In [7]:
lb_100000.df

,"h_50, b_1.5, p_14, ATR","h_50, b_1.5, p_14, V","h_50, b_1.5, p_28, ATR","h_50, b_1.5, p_28, V","h_50, b_2, p_14, ATR","h_50, b_2, p_14, V","h_50, b_2, p_28, ATR","h_50, b_2, p_28, V","h_100, b_1.5, p_14, ATR","h_100, b_1.5, p_14, V","h_100, b_1.5, p_28, ATR","h_100, b_1.5, p_28, V","h_100, b_2, p_14, ATR","h_100, b_2, p_14, V","h_100, b_2, p_28, ATR","h_100, b_2, p_28, V"
datetime,,,,,,,,,,,,,,,,
2017-01-02 17:03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-03 08:31:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-03 09:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-03 09:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-03 10:12:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-19 12:18:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-19 13:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-19 14:46:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Chequeo balance entre etiquetas

In [8]:
for column in lb_100000.df.columns:
    print(lb_100000.df[column].value_counts(dropna=False))

h_50, b_1.5, p_14, ATR
 1.0    6711
-1.0    5927
 0.0      57
 NaN      50
Name: count, dtype: int64
h_50, b_1.5, p_14, V
 1.0    6733
-1.0    5934
 NaN      50
 0.0      28
Name: count, dtype: int64
h_50, b_1.5, p_28, ATR
 1.0    6734
-1.0    5884
 0.0      77
 NaN      50
Name: count, dtype: int64
h_50, b_1.5, p_28, V
 1.0    6725
-1.0    5934
 NaN      50
 0.0      36
Name: count, dtype: int64
h_50, b_2, p_14, ATR
 1.0    6693
-1.0    5734
 0.0     268
 NaN      50
Name: count, dtype: int64
h_50, b_2, p_14, V
 1.0    6696
-1.0    5949
 0.0      50
 NaN      50
Name: count, dtype: int64
h_50, b_2, p_28, ATR
 1.0    6737
-1.0    5688
 0.0     270
 NaN      50
Name: count, dtype: int64
h_50, b_2, p_28, V
 1.0    6670
-1.0    5968
 0.0      57
 NaN      50
Name: count, dtype: int64
h_100, b_1.5, p_14, ATR
 1.0    6719
-1.0    5912
 NaN     100
 0.0      14
Name: count, dtype: int64
h_100, b_1.5, p_14, V
 1.0    6715
-1.0    5915
 NaN     100
 0.0      15
Name: count, dtype: int64
h_100,

In [9]:
len(vv_100000), len(lb_100000.df), len(ff_100000)

(12745, 12745, 12745)